In [101]:
import pandas as pd
import glob
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score

In [75]:
# Read and concatenate all CSV files
df = pd.read_csv("DrDoS_DNS_data_1_per.csv")

/var/folders/rh/s8rpwcgj7zd5btb07r6f891m0000gn/T/ipykernel_1450/477989729.py:2: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("DrDoS_DNS_data_1_per.csv")


In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5074413 entries, 0 to 5074412
Data columns (total 88 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   Unnamed: 0                    int64  
 1   Flow ID                       object 
 2    Source IP                    object 
 3    Source Port                  int64  
 4    Destination IP               object 
 5    Destination Port             int64  
 6    Protocol                     int64  
 7    Timestamp                    object 
 8    Flow Duration                int64  
 9    Total Fwd Packets            int64  
 10   Total Backward Packets       int64  
 11  Total Length of Fwd Packets   float64
 12   Total Length of Bwd Packets  float64
 13   Fwd Packet Length Max        float64
 14   Fwd Packet Length Min        float64
 15   Fwd Packet Length Mean       float64
 16   Fwd Packet Length Std        float64
 17  Bwd Packet Length Max         float64
 18   Bwd Packet Length Min

In [77]:
filtered_data = df.drop(columns=['Unnamed: 0', 'Flow ID', ' Source IP', ' Source Port', ' Destination IP', ' Destination Port', ' Timestamp', 'SimillarHTTP' ], axis=1)
filtered_data.rename(columns={' Label': 'Label'}, inplace=True)
filtered_data.rename(columns={' Protocol': 'Protocol'}, inplace=True)
filtered_data.head()

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Inbound,Label
0,17,28415,97,0,42680.0,0.0,440.0,440.0,440.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,DrDoS_DNS
1,17,2,2,0,880.0,0.0,440.0,440.0,440.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,DrDoS_DNS
2,17,48549,200,0,88000.0,0.0,440.0,440.0,440.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,DrDoS_DNS
3,17,48337,200,0,88000.0,0.0,440.0,440.0,440.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,DrDoS_DNS
4,17,32026,200,0,88000.0,0.0,440.0,440.0,440.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,DrDoS_DNS


In [78]:
filtered_data.Label.value_counts()

Label
DrDoS_DNS    5071011
BENIGN          3402
Name: count, dtype: int64

In [79]:
filtered_data.Protocol.value_counts()

Protocol
17    5070662
6        3369
0         382
Name: count, dtype: int64

In [80]:
# One-hot encoding
df_one_hot = pd.get_dummies(filtered_data, columns=['Protocol'], prefix='Protocol')

df_one_hot.Protocol_6.value_counts()


Protocol_6
False    5071044
True        3369
Name: count, dtype: int64

In [81]:
X = filtered_data.drop('Label', axis=1)

In [82]:
X.isnull().sum()
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5074413 entries, 0 to 5074412
Data columns (total 79 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   Protocol                      int64  
 1    Flow Duration                int64  
 2    Total Fwd Packets            int64  
 3    Total Backward Packets       int64  
 4   Total Length of Fwd Packets   float64
 5    Total Length of Bwd Packets  float64
 6    Fwd Packet Length Max        float64
 7    Fwd Packet Length Min        float64
 8    Fwd Packet Length Mean       float64
 9    Fwd Packet Length Std        float64
 10  Bwd Packet Length Max         float64
 11   Bwd Packet Length Min        float64
 12   Bwd Packet Length Mean       float64
 13   Bwd Packet Length Std        float64
 14  Flow Bytes/s                  float64
 15   Flow Packets/s               float64
 16   Flow IAT Mean                float64
 17   Flow IAT Std                 float64
 18   Flow IAT Max         

In [83]:
columns_with_inf = X.columns[np.isinf(X).any(axis=0)]

print(f"Columns with infinity: {columns_with_inf.tolist()}")

Columns with infinity: ['Flow Bytes/s', ' Flow Packets/s']


In [84]:
X = np.where(np.isinf(X), np.nan, X) 
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

In [85]:
minMaxScaler = MinMaxScaler()
X_normalized = minMaxScaler.fit_transform(X)
X_normalized[:10]

array([[1.00000000e+00, 2.36791673e-04, 9.58590871e-04, 0.00000000e+00,
        2.79567909e-03, 0.00000000e+00, 1.36986301e-02, 2.17821782e-01,
        1.45922920e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 5.10198227e-04, 8.53412672e-04,
        6.56936670e-06, 8.52438172e-06, 2.99820490e-05, 2.21945875e-08,
        2.36791888e-04, 6.56936670e-06, 8.52438172e-06, 2.99820490e-05,
        2.21945875e-08, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 9.99999383e-01, 9.99930711e-01,
        8.53422488e-04, 0.00000000e+00, 2.98913043e-01, 1.31030375e-02,
        1.65272789e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.66903429e-01, 1.45922920e-01, 0.00000000e+00, 9.999993

In [86]:
label_mapping = {'BENIGN': 1, 'DrDoS_DNS': 2}
filtered_data['Label'] = filtered_data['Label'].map(label_mapping)
y = filtered_data['Label']

In [87]:
X_train, X_temp, y_train, y_temp = train_test_split(X_normalized, y, test_size=0.4, stratify=y)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp)


In [88]:
values, counts = np.unique(y_valid, return_counts=True)
print(values, counts)

[1 2] [    681 1014202]


In [89]:
class myDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float64)
        self.y = torch.tensor(y.values if isinstance(y, pd.Series) else y, dtype=torch.long)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [91]:
train_dataset = myDataset(X_train, y_train)
valid_dataset = myDataset(X_valid, y_valid)
test_dataset = myDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
val_loader = DataLoader(valid_dataset, batch_size=10, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

In [93]:
for X, y in train_loader:
    print(X.shape)
    print(y.shape)
    break

torch.Size([10, 79])
torch.Size([10])


In [95]:
class FFNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes=2): # numclass 2 because classified benign and non benigm=n, hidden size = jumlah neuron, # input size = jumlah feature
        super(FFNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) # layer 1
        self.fc2 = nn.Linear(hidden_size, num_classes) # layer 2
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        #x = self.relu(self.fc2(x))
        # x = self.fc3(x)
        return x

In [96]:
input_size = X_train.shape[1]
num_classes = 2
hidden_size = 32
model = FFNN(input_size, hidden_size, num_classes)

In [97]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
# optimizer = optim.Adam(model.parameters())
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
criterion = nn.CrossEntropyLoss()

In [99]:
from torch.utils.tensorboard import SummaryWriter
import datetime

In [100]:
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [103]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = f'runs/FFNN_{current_time}'
writer = SummaryWriter(log_dir)

early_stopping = EarlyStopping(patience=5, verbose=True)

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    all_labels = []
    all_predictions = []
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())
    
    avg_loss = total_loss / len(train_loader)
    train_accuracy = 100 * np.mean(np.array(all_labels) == np.array(all_predictions))
    train_f1 = f1_score(all_labels, all_predictions, average='weighted')
    
    # Log training metrics
    writer.add_scalar('Loss/Train', avg_loss, epoch)
    writer.add_scalar('Accuracy/Train', train_accuracy, epoch)
    writer.add_scalar('F1 Score/Train', train_f1, epoch)
    
    # Validation step
    model.eval()
    val_loss = 0
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
        
        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = 100 * np.mean(np.array(all_labels) == np.array(all_predictions))
        val_f1 = f1_score(all_labels, all_predictions, average='weighted')
        
        # Log validation metrics
        writer.add_scalar('Loss/Validation', avg_val_loss, epoch)
        writer.add_scalar('Accuracy/Validation', val_accuracy, epoch)
        writer.add_scalar('F1 Score/Validation', val_f1, epoch)
    
    # Log learning rate
    current_lr = optimizer.param_groups[0]['lr']
    writer.add_scalar('Learning Rate', current_lr, epoch)
    
    # if epoch % 10 == 0:
    print(f'Epoch [{epoch}/{num_epochs}], Train Loss: {avg_loss:.4f}, Train Acc: {train_accuracy:.2f}%, Train F1: {train_f1:.4f}, '
            f'Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%, Val F1: {val_f1:.4f}, LR: {current_lr:.6f}')
    early_stopping(avg_val_loss, model)
    if early_stopping.early_stop:
        print("Early stopping")
        break

    if scheduler:
        scheduler.step()

#model.load_state_dict(torch.load('checkpoint.pt'))

writer.close()

RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float